In [1]:
import requests,json,psycopg2
import pandas as pd
import numpy as np

In [169]:
def getVipProjectById(projectId):
    conn = psycopg2.connect("dbname='CRM' user='read_only' host='192.168.10.10' password='User_ro'")
    cur = conn.cursor()
    cur.execute("SELECT p.projectid, p.domain, p.status, p2.login, p2.password FROM project AS p INNER JOIN ProjectMetrikaData AS p2 ON p.UniqueId = p2.ProjectUniqueId WHERE projectid=%s AND p.status<230 ORDER BY p.projectid", [projectId])
    result = []
    rows = cur.fetchall()
    for row in rows:
        result.append(getVipProjectByRow(row))
    return result
def getVipProjectByRow(row):
    return {'projectid': row[0], 'domain': row[1], 'metrikalogin': row[3], 'metrikapassword': row[4]}
def GetToken(login,serviceType):
    if serviceType not in ['Metrika','Direct']:
        print 'Invalid service type'
        return None
    url = 'http://192.168.10.37:35678/TokenManagerService'
    payload = {'method':'GetToken','params':{'login':login.replace('@yandex.ru',''),'serviceType':serviceType}}
    r = requests.post(url,json.dumps(payload))
    data = json.loads(r.text)
    if len(data.keys()) == 1:
        print 'Invalid login'
        return None
    return data['result']['token']
def getMetrikaCountersByToken(oAuthToken): # Получаем массив словарей со списком счетчиков для токена
    url = 'https://beta.api-metrika.yandex.ru/management/v1/counters?oauth_token={0}'.format(oAuthToken)
    r = requests.get(url)
    try:
        response = json.loads(r.text)
        counters = response['counters']
        return counters
    except:
        return '-1'
def getMetrikaCounterIDByDomain(counters, domain): # Получаем ID счетчика для домена
    counterID = []
    for counter in counters:
        try: # Иногда отваливается с ошибкой string indices must be integers. Возможное решение: вызывать несколько раз getMetrikaCountersByToken http://stackoverflow.com/questions/7293978/repeat-an-iteration-of-for-loop
            if domain in counter['site'] and counter['code_status'] == 'CS_OK':
                counterID.append(counter['id'])
        except:
            print 'getMetrikaCounterIDByDomain exception'
            continue
    return counterID
def getExtendedProjectInfo(projectId,serviceType):
    vipProjectInfo = getVipProjectById(projectId)[0]
    token = GetToken(vipProjectInfo['metrikalogin'],serviceType)
    if serviceType == 'Metrika':
        counters = getMetrikaCountersByToken(token)
        counterId = getMetrikaCounterIDByDomain(counters, vipProjectInfo['domain'])[0]
        vipProjectInfo['counterId'] = counterId
    vipProjectInfo['token'] = token
    return vipProjectInfo
def getMetrikaGoalMetrics(projectid):
    goalsMetrics = ','.join(['ym:s:goal'+str(target['targetid'])+'visits' for target in getCPATargetsByProjectID(projectid)])
    return goalsMetrics
def getCPATargetsByProjectID(projectID):
    conn = psycopg2.connect("dbname='Cubo' user='read_only' host='192.168.10.8' password='User_ro'")
    cur = conn.cursor()
    cur.execute("SELECT targetstatus, targetid, targetname FROM metricstarget WHERE projectid=%s ORDER BY id DESC", [projectID])
    result = []
    rows = cur.fetchall()
    for row in rows:
        result.append(getCPATargetByRow(row))
    return result
def getCPATargetByRow(row):
    return {'targetstatus': row[0], 'targetid': row[1], 'targetname': row[2]}
def getMetrikaGoalMetrics(projectid):
    goalsMetrics = ','.join(['ym:s:goal'+str(target['targetid'])+'visits' for target in getCPATargetsByProjectID(projectid)])
    return goalsMetrics
def getMetrikaReportInDataFrame(date1,date2,metrics,dimensions,token,counterId,filters,attribution,limit):
    params = '&'.join(['dimensions={0}'.format(dimensions),
                       'metrics={0}'.format(metrics),
                       'date1={0}'.format(date1),
                      'date2={0}'.format(date2),
                      'ids={0}'.format(counterId),
                      'oauth_token={0}'.format(token),
                      'limit={0}'.format(limit),
                      'offset=1',
                      "filters={0}".format(filters),
                      'accuracy=full',
                      'attribution={0}'.format(attribution)])
    url = 'https://beta.api-metrika.yandex.ru/stat/v1/data.csv?'+params
    print url
    df = pd.read_csv(url,encoding='utf-8')
    return df
def dropTotalRow(df):
    return df.drop(df.index[0]).reset_index(drop=True)
def mergeCPAVisitsColumns(df):
    for i in xrange(0,len(df)):
        df.loc[i,'goal_visits'] = df.loc[i,[column for column in df.columns if u'Целевые визиты' in column]].sum()
    df.drop([column for column in df.columns if u'Целевые визиты' in column], axis=1, inplace=True)
    return df
def getDirectCallByProjectID(projectID,date1,date2):
    conn = psycopg2.connect("dbname='CuboHistory' user='read_only' host='192.168.10.32' password='User_ro'")
    cur = conn.cursor()
    cur.execute("SELECT id, duration, phonenumbera, date FROM calltrackinginfo where projectid={0} and campaign='Поисковое продвижение' and date >= '{1}' and date <= '{2}'".format(projectID,date1,date2))
    result = []
    rows = cur.fetchall()
    for row in rows:
        result.append(getCallByRow(row))
    return result
def getCallByRow(row):
    return {'id': row[0], 'duration': row[1], 'phonenumbera': row[2], 'date':row[3]}

In [173]:
projectId = 2410910
date1 = '2016-07-01'
date2 = '2016-07-31'

extendedProjectInfo = getExtendedProjectInfo(projectId,'Metrika')
goalMetrics = getMetrikaGoalMetrics(projectId)
ids = extendedProjectInfo['counterId']
token = extendedProjectInfo['token']
attribution = 'lastsign'

limit = 100
metrics = 'ym:s:visits,ym:s:bounceRate,ym:s:pageDepth,ym:s:avgVisitDurationSeconds'

# Sources Report No. 1
dimensions = 'ym:s:<attribution>TrafficSource'
sources_report_1 = getMetrikaReportInDataFrame(date1,date2,metrics,dimensions,token,ids,'',attribution,limit)
sources_report_1.rename(columns={u'﻿"Источник трафика"':u'Источник трафика'},inplace=True)

# Sources Report No. 2
dimensions = 'ym:s:<attribution>SearchEngineRoot'
sources_report_2 = getMetrikaReportInDataFrame(date1,date2,metrics,dimensions,token,ids,'',attribution,limit)
sources_report_2.rename(columns={u'﻿"Поисковая система"':u'Поисковая система'},inplace=True)

# Keywords Report
limit = 20
dimensions = 'ym:s:<attribution>SearchPhrase'
filters = "ym:s:<attribution>TrafficSource=='organic'"
keywords_report = getMetrikaReportInDataFrame(date1,date2,metrics,dimensions,token,ids,filters,attribution,limit)
keywords_report.rename(columns={u'﻿"Поисковая фраза"':u'Поисковая фраза'},inplace=True)
keywords_report = dropTotalRow(keywords_report)

# Start URLs Report
limit = 10
dimensions = 'ym:s:startURL'
filters = "ym:s:<attribution>TrafficSource=='organic'"
start_urls_report = getMetrikaReportInDataFrame(date1,date2,metrics,dimensions,token,ids,filters,attribution,limit)
start_urls_report.rename(columns={u'﻿"Страница входа"':u'Страница входа'},inplace=True)
start_urls_report = dropTotalRow(start_urls_report)

# Geo Report
limit = 15
dimensions = 'ym:s:regionCountryName,ym:s:regionAreaName'
filters = "ym:s:<attribution>TrafficSource=='organic'"
geo_report = getMetrikaReportInDataFrame(date1,date2,metrics,dimensions,token,ids,filters,attribution,limit)
geo_report.rename(columns={u'﻿"Страна"':u'Страна'},inplace=True)

# Goal Report
if goalMetrics != '':
    limit = 100000
    dimensions = 'ym:s:date,ym:s:<attribution>SearchEngineRoot'
    filters = "ym:s:<attribution>TrafficSource=='organic'"
    metrics = goalMetrics
    goal_report = getMetrikaReportInDataFrame(date1,date2,metrics,dimensions,token,ids,filters,attribution,limit)
    goal_report.rename(columns={u'﻿"Дата визита"':u'Дата визита'},inplace=True)
    goal_report = mergeCPAVisitsColumns(dropTotalRow(goal_report))
    pivot_goal_report = pd.pivot_table(goal_report, values='goal_visits', index=[u'Дата визита'],columns=[u'Поисковая система'], aggfunc=np.sum, fill_value=0.0,margins=False)
else:
    print 'There is no goal metrics!'
    
# Calls Report
calls_report = pd.DataFrame(getDirectCallByProjectID(projectId,date1,date2))
if len(calls_report) > 0:
    calls_report['date2'] = calls_report['date'].dt.date
    grouped_calls_report = calls_report[['date2','duration']].groupby(['date2'],as_index=False).count()
    grouped_calls_report.columns=[u'Дата',u'Звонки']
else:
    print 'There is no calls!'

from pandas import ExcelWriter
from openpyxl import load_workbook
book = load_workbook(u'D:\\_Docs_\\Копия отчет по SEO_диаграммы и графики.xlsx')
sheetName = 'Data'
ws = book.get_sheet_by_name(sheetName)
if ws is not None:
    book.remove_sheet(ws)
writer = ExcelWriter(u'D:\\_Docs_\\Копия отчет по SEO_диаграммы и графики.xlsx', engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
start_row = 0
sources_report_1.to_excel(writer, "Data",startrow=start_row)
start_row = start_row + len(sources_report_1) + 2
sources_report_2.to_excel(writer, "Data",startrow=start_row)
start_row = start_row + len(sources_report_2) + 2
keywords_report.to_excel(writer, "Data",startrow=start_row)
start_row = start_row + len(keywords_report) + 2
start_urls_report.to_excel(writer, "Data",startrow=start_row)
start_row = start_row + len(start_urls_report) + 2
geo_report.to_excel(writer, "Data",startrow=start_row)
start_row = start_row + len(geo_report) + 2
if goalMetrics != '':
    pivot_goal_report.to_excel(writer, "Data",startrow=start_row)
    start_row = start_row + len(pivot_goal_report) + 2
if len(calls_report) > 0:
    grouped_calls_report.to_excel(writer, "Data",startrow=start_row)
    start_row = start_row + len(grouped_calls_report) + 2
writer.save()

https://beta.api-metrika.yandex.ru/stat/v1/data.csv?dimensions=ym:s:<attribution>TrafficSource&metrics=ym:s:visits,ym:s:bounceRate,ym:s:pageDepth,ym:s:avgVisitDurationSeconds&date1=2016-07-01&date2=2016-07-31&ids=7820509&oauth_token=AQAAAAAHnjNRAAFxmP5hRPFrFEQzjcB6TxbqIvg&limit=100&offset=1&filters=&accuracy=full&attribution=lastsign
https://beta.api-metrika.yandex.ru/stat/v1/data.csv?dimensions=ym:s:<attribution>SearchEngineRoot&metrics=ym:s:visits,ym:s:bounceRate,ym:s:pageDepth,ym:s:avgVisitDurationSeconds&date1=2016-07-01&date2=2016-07-31&ids=7820509&oauth_token=AQAAAAAHnjNRAAFxmP5hRPFrFEQzjcB6TxbqIvg&limit=100&offset=1&filters=&accuracy=full&attribution=lastsign
https://beta.api-metrika.yandex.ru/stat/v1/data.csv?dimensions=ym:s:<attribution>SearchPhrase&metrics=ym:s:visits,ym:s:bounceRate,ym:s:pageDepth,ym:s:avgVisitDurationSeconds&date1=2016-07-01&date2=2016-07-31&ids=7820509&oauth_token=AQAAAAAHnjNRAAFxmP5hRPFrFEQzjcB6TxbqIvg&limit=20&offset=1&filters=ym:s:<attribution>TrafficSo

In [13]:
login = 'root'
password = '1ldlNQiVnL'
yt_url = 'https://cubo.myjetbrains.com/youtrack'
login_url = '/rest/user/login'
cookies = dict(login=login,password=password)
requests.post('{0}{1}'.format(yt_url,login_url))

<Response [400]>

In [5]:
cookies

{'login': 'root', 'password': '1ldlNQiVnL'}

In [29]:
import httplib2,urllib
class Connection(object):
    def __init__(self, url, login=None, password=None, proxy_info=None, api_key=None):
        self.http = httplib2.Http(disable_ssl_certificate_validation=True) if proxy_info is None else httplib2.Http(
            proxy_info=proxy_info, disable_ssl_certificate_validation=True)

        # Remove the last character of the url ends with "/"
        if url:
            url = url.rstrip('/')

        self.url = url
        self.baseUrl = url + "/rest"
        if api_key is None:
            self._credentials = (login, password)
            self._login(*self._credentials)
        else:
            self.headers = {'X-YouTrack-ApiKey': api_key}

    def _login(self, login, password):
        response, content = self.http.request(
            self.baseUrl + "/user/login?login=" + urllib.quote_plus(login) + "&password=" + urllib.quote_plus(password),
            'POST',
            headers={'Content-Length': '0', 'Connection': 'keep-alive'})
        if response.status != 200:
            raise youtrack.YouTrackException('/user/login', response, content)
        self.headers = {'Cookie': response['set-cookie'],
                        'Cache-Control': 'no-cache'}
    
    def _req(self, method, url, body=None, ignoreStatus=None, content_type=None):
        headers = self.headers
        if method == 'PUT' or method == 'POST':
            headers = headers.copy()
            if content_type is None:
                content_type = 'application/xml; charset=UTF-8'
            headers['Content-Type'] = content_type
            headers['Content-Length'] = str(len(body)) if body else '0'

        response, content = self.http.request((self.baseUrl + url).encode('utf-8'), method, headers=headers, body=body)
        content = content.translate(None, '\0')
        return response, content

    def _reqXml(self, method, url, body=None, ignoreStatus=None):
        response, content = self._req(method, url, body, ignoreStatus)
        if response.has_key('content-type'):
            if (response["content-type"].find('application/xml') != -1 or response["content-type"].find(
                'text/xml') != -1) and content is not None and content != '':
                try:
                    return minidom.parseString(content)
                except Exception:
                    return ""
            elif response['content-type'].find('application/json') != -1 and content is not None and content != '':
                try:
                    return json.loads(content)
                except Exception:
                    return ""

        if method == 'PUT' and ('location' in response.keys()):
            return 'Created: ' + response['location']
        else:
            return content

    def _get(self, url):
        return self._reqXml('GET', url)
    


In [20]:
yt = Connection('https://cubo.myjetbrains.com/youtrack/', 'root', '1ldlNQiVnL')

In [30]:
yt._get('https://cubo.myjetbrains.com/youtrack/rest/issue/SEO_MM-724/link')

AttributeError: 'Connection' object has no attribute '_req'

In [31]:
def cumulative_rate_monthly_saving(start_amount,
                                   yearly_rate,
                                   months_saving,
                                   monthly_saving):
    if months_saving == 0:
        return start_amount
    monthly_rate = 1 + yearly_rate/12
    return (cumulative_rate_monthly_saving(start_amount,
                                           yearly_rate,
                                           months_saving-1,
                                           monthly_saving)
            + monthly_saving) * monthly_rate


In [36]:
cumulative_rate_monthly_saving(100000,0.08,5*12,40000)

3107652.6489718854

In [45]:
def function(data=[]):
    data.append(1)
    return data

In [38]:
function()

[1]

In [39]:
function

<function __main__.function>

In [54]:
function([2])

[2, 1]